In [1]:
import os
from glob import glob
import sys
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

sys.path.insert(0, '../')
from dataset import REDataset, get_train_test_loader
from models import load_model
from tokenization import load_tokenizer
from config import Config, ModelType, PreTrainedType, TokenizationType

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
model = load_model()

Load Model...	Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized fro

In [4]:
dataset = REDataset()

Load Tokenizer...	done!
Load raw data...	done!
Apply Tokenization...	done!


In [5]:
train_loader, valid_loader = get_train_test_loader(dataset)

In [6]:
for sentence, label in train_loader:
    break

In [23]:
tokenized_data = tokenizer(
            raw["relation_state"].tolist(),
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=100,
            add_special_tokens=True,
        )

In [24]:
tokenized_data

{'input_ids': tensor([[  101, 50266, 11489,  ...,     0,     0,     0],
        [  101,  9428, 41521,  ...,     0,     0,     0],
        [  101, 68495, 37905,  ...,     0,     0,     0],
        ...,
        [  101,  9328, 20309,  ...,     0,     0,     0],
        [  101, 49780, 16617,  ...,     0,     0,     0],
        [  101,  9665, 43852,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}